# Esercizio 1 
#### Input e download 

In [62]:
import nltk
import random
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import movie_reviews
from nltk.corpus import stopwords, words
from nltk.corpus.util import LazyCorpusLoader
from nltk.stem import PorterStemmer
from nltk.corpus.reader import *
from nltk.stem import WordNetLemmatizer
import re
from nltk.metrics.scores import precision, recall, f_measure
import numpy as np
from collections import defaultdict
from nltk.tokenize import word_tokenize
#nltk.download('europarl_raw')
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('omw-1.4')
#nltk.download('punkt')

In [63]:
def find_features(document):
    words = set(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features

### Import the raw of EuroparlCorpusReader of different language
save **800000** words in **english** and **200000** words **for each other language**

In [64]:
english = LazyCorpusLoader('europarl_raw/english', EuroparlCorpusReader, r'ep-.*\.en', encoding='utf-8')
italian =  LazyCorpusLoader('europarl_raw/italian', EuroparlCorpusReader, r'ep-.*\.it', encoding='utf-8')
french =   LazyCorpusLoader('europarl_raw/french', EuroparlCorpusReader, r'ep-.*\.fr', encoding='utf-8')
spanish = LazyCorpusLoader('europarl_raw/spanish', EuroparlCorpusReader, r'ep-.*\.es', encoding='utf-8')
german = LazyCorpusLoader('europarl_raw/german', EuroparlCorpusReader, r'ep-.*\.de', encoding='utf-8')

words = english.raw()[:800000]
words += italian.raw()[:200000]
words += french.raw()[:200000]
words += spanish.raw()[:200000]
words += german.raw()[:200000]
tokenizer = RegexpTokenizer(r'\w+')
words=tokenizer.tokenize(words)

len(words)

252605

Make a list of stopword 

In [65]:
stop_words_list = stopwords.words('english')
stop_words_list += stopwords.words('italian')
stop_words_list += stopwords.words('french')
stop_words_list += stopwords.words('spanish')
stop_words_list += stopwords.words('german')
stop_words_list = set(stop_words_list)


#### Reduce the BoW removing the **stopword** and **use lemmatizer and stemming** for reduce the words to the **roots** after this make a shuffle of the list

In [66]:
lemmatizer = WordNetLemmatizer()
stemmer= PorterStemmer()
filtered_sentence = []
for w in words:
    s = stemmer.stem(w)
    w = lemmatizer.lemmatize(s)
    if w not in stop_words_list :
        filtered_sentence.append(w)
np.random.shuffle(filtered_sentence)


Calculate the **frequency distribution** of the words in the corpus

In [67]:
word_frequency_distribution = nltk.FreqDist([word.lower() for word in filtered_sentence])

len(filtered_sentence)

133091

Take only the first **8000 words most used**

In [68]:
word_features = [i[0] for i in word_frequency_distribution.most_common(8000)]
word_features2 = list(word_frequency_distribution.keys())[:8000]



Create **featuresets** taken **8000 sentences for english** and **2000 for each of other language**

In [69]:
featuresets = [ (find_features(sentence),"eng" ) for sentence in english.sents()[:8000]]
featuresets += [ (find_features(sentence),"not eng" ) for sentence in italian.sents()[:2000]]
featuresets += [ (find_features(sentence),"not eng" ) for sentence in french.sents()[:2000]]
featuresets += [ (find_features(sentence),"not eng" ) for sentence in spanish.sents()[:2000]]
featuresets += [ (find_features(sentence),"not eng" ) for sentence in german.sents()[:2000]]
np.random.shuffle(featuresets)


Split featuresets in **80:20** and obtain **Training and Testing**

In [70]:
training = featuresets[0:int(len(featuresets) * 0.8)]
testing = featuresets[int(len(featuresets) * 0.8):]

Train my classifier with training_set

In [71]:
classifier = nltk.NaiveBayesClassifier.train(training)

## Make all the metric
#### Accuracy:

In [72]:
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing))*100)

Classifier accuracy percent: 96.4375


#### Precision, Recall:

In [73]:
predictions, gold_labels = defaultdict(set), defaultdict(set)

for i, (features, label) in enumerate(testing):
    predictions[classifier.classify(features)].add(i)
    gold_labels[label].add(i) 

for label in predictions:
    print(label, 'Precision:', precision(gold_labels[label], predictions[label]))
    print(label, 'Recall:', recall(gold_labels[label], predictions[label]))
    print(label, 'f1_measure:', f_measure(gold_labels[label], predictions[label]))
    print()

not eng Precision: 0.9986541049798116
not eng Recall: 0.9298245614035088
not eng f1_measure: 0.963011031797534

eng Precision: 0.9346557759626605
eng Recall: 0.9987531172069826
eng f1_measure: 0.9656419529837251



#### Confusion Matrix

In [74]:
test_result = []
gold_result = []

for i in range(len(testing)):
    test_result.append(classifier.classify(testing[i][0]))
    gold_result.append(testing[i][1])
CM = nltk.ConfusionMatrix(gold_result, test_result)
print(CM)

        |         n |
        |         o |
        |         t |
        |           |
        |    e    e |
        |    n    n |
        |    g    g |
--------+-----------+
    eng |<1602>   2 |
not eng |  112<1484>|
--------+-----------+
(row = reference; col = test)



### Show most informative features of classifier

In [75]:
classifier.show_most_informative_features(25)

Most Informative Features
                  report = True              eng : not en =    203.9 : 1.0
                    time = True              eng : not en =    141.8 : 1.0
                    area = True              eng : not en =     81.8 : 1.0
                  regard = True              eng : not en =     69.1 : 1.0
                    dire = True           not en : eng    =     45.6 : 1.0
                    être = True           not en : eng    =     44.1 : 1.0
                     law = True              eng : not en =     37.0 : 1.0
              commission = True           not en : eng    =     31.0 : 1.0
                  common = True              eng : not en =     30.2 : 1.0
                   prima = True           not en : eng    =     27.6 : 1.0
                 mention = True              eng : not en =     27.0 : 1.0
                 opinion = True              eng : not en =     26.2 : 1.0
                  market = True              eng : not en =     24.1 : 1.0

# Test the classifier

In [76]:
def test(text):
    text= word_tokenize(text)
    feat = find_features(text)
    return classifier.classify(feat)

### Test the Classifier passing String in different language
##### English

In [77]:

print(test("The test of the classifier shall give out the measures of accuracy, precision, recall on the obtained confusion matrix and WILL NOT BE EVALUATED ON THE LEVEL OF THE PERFORMANCES.\
 In other terms, when the confusion is produced, then the value of the assignment will be good, independently of the percentage of false positive and negative results."))

eng


##### Italian

In [78]:
print(test("Il test del classificatore darà le misure di accuratezza, precisione, richiamo sulla matrice di confusione ottenuta e NON SARÀ VALUTATO SUL LIVELLO DELLE PRESTAZIONI.\
     In altri termini, quando si produrrà la confusione, allora il valore dell'incarico sarà buono, indipendentemente dalla percentuale di risultati falsi positivi e negativi."))

not eng


##### French

In [79]:
print(test("Le test du classificateur doit donner les mesures d'exactitude, de précision, de rappel sur la matrice de confusion obtenue et NE SERA PAS ÉVALUÉ AU NIVEAU DES PERFORMANCES.\
     En d'autres termes, lorsque la confusion est produite, alors la valeur de l'affectation sera bonne, indépendamment du pourcentage de résultats faux positifs et négatifs."))

not eng


##### Spanish

In [80]:
print(test("La prueba del clasificador deberá dar las medidas de exactitud, precisión, recordación sobre la matriz de confusión obtenida y NO SERÁ EVALUADA EN EL NIVEL DE LAS ACTUACIONES.\
     En otros términos, cuando se produzca la confusión, entonces el valor de la asignación será bueno, independientemente del porcentaje de resultados falsos positivos y negativos."))

not eng


##### German

In [81]:
print(test("Der Test des Klassifikators gibt die Maße für Genauigkeit, Präzision und Wiedererinnerung an der erhaltenen Konfusionsmatrix an und WIRD NICHT AUF DER NIVEAU DER LEISTUNGEN \
    BEWERTET. Mit anderen Worten, wenn die Verwirrung zustande kommt, dann ist der Wert der Zuordnung gut, unabhängig vom Prozentsatz falsch positiver und negativer Ergebnisse."))

not eng
